In [339]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from scipy import stats

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from utilities.plotting_template import local_theme

pio.templates.default = local_theme


file = "data/probability-distribution-of-tunnel-cost-overuns-cdf.csv"

# Load the data
df = pd.read_csv(file)

# Plot the CDF as shown in the report
fig = px.scatter(df, x="OverrunPercent", y="CDF").update(
    layout=dict(
        title="<b>Distribution of Cost Overrunns</b><br><i>Fixed Links (bridges and tunnel)</i>",
        height=500,
        width=800,
        xaxis=dict(
            title="Cost overrun vs budget",
            range=[-0.2, 1.8],
            dtick=0.2,
        ),
        yaxis=dict(
            title="Share of projects with given<br>max. cost overrun",
            range=[0, 1],
            dtick=0.2,
            tickformat="{:.0}",
        ),
        margin=dict(t=60, b=20, l=20, r=20),
    )
)

fig.write_image("probability-distribution-of-tunnel-cost-overruns-cdf-original.svg")
fig.show()

In [340]:
# Convert the cost overrun percent to a cost factor
# so that the x values are not negative (helps with the math)
df["CostFactor"] = 1 + df["OverrunPercent"]
df = df[["CostFactor", "CDF"]]

# Plot the cost factor
fig_cdf = px.scatter(df, x="CostFactor", y="CDF").update(
    layout=dict(
        title="<b>Distribution of Cost Factors</b><br><i>Fixed Links (bridges and tunnel)</i>",
        height=500,
        width=800,
        xaxis=dict(title="Cost factor", range=[0.8, 2.8], dtick=0.2),
        yaxis=dict(
            title="Share of projects with given<br>cost factor",
            range=[0, 1],
            dtick=0.2,
        ),
        margin=dict(t=60, b=20, l=20, r=20),
    )
)
fig_cdf.show()

In [341]:
# The CDF in the paper does not extent from 0 to 1 but from ~0.1 to 0.95
# To run the rest of the process, it should be extended to the range between 0 and 1
# Also, non-increasing portions of the chart should be removed

# Remove the portions of the chart that are not increasing
df = df.loc[
    (df["CDF"].diff() > 0)
    & ((df["CostFactor"].diff().notnull()) & (df["CostFactor"].diff() > 0))
]

# Get the x-intercept (where y == 0) for the plot
# For the linear regression, use the `CDF` as the x value and `CostFactor` as the y value
# These are opposite from the plot, so they give the y-int instead of the x-int
slope, intercept = stats.linregress(df.loc[0:5][["CDF", "CostFactor"]])[0:2]

# Add the intercept as a point in the dataframe
df1 = pd.DataFrame([{"CDF": 0, "CostFactor": intercept}, {"CDF": 1, "CostFactor": 2.8}])
df = (
    pd.concat(
        [
            df,
            df1,
        ],
        ignore_index=True,
    )
    .sort_values("CDF")
    .reset_index()
    .drop(columns="index")
)

# See what the new plot looks like
# Plot the cost factor
fig_cdf = px.scatter(df, x="CostFactor", y="CDF").update(
    layout=dict(
        title="<b>Distribution of Cost Factors</b><br><i>Fixed Links (bridges and tunnel)</i>",
        height=500,
        width=800,
        xaxis=dict(title="Cost factor", range=[0.8, 2.8], dtick=0.2),
        yaxis=dict(
            title="Share of projects with given<br>cost factor",
            range=[0, 1],
            dtick=0.2,
        ),
        margin=dict(t=60, b=20, l=20, r=20),
    )
)

fig_cdf.write_image(
    "probability-distribution-of-tunnel-cost-overruns-cdf-costfactor.svg"
)
fig_cdf.show()

In [342]:
# Add the upper and lower bounds to the CDF
# These were not included in the original image, but help with the processing
def inverse_cdf(u: float) -> float:
    "Perform a linear interpolation using `CDF` as the `x` and `CostFactor` as the `y`"
    return np.interp(u, df["CDF"], df["CostFactor"])


df = df.sort_values("CDF")
# Generate N uniform random numbers
N = 10000  # for example
uniform_random_numbers = np.random.uniform(0, 1, N)

# Apply the inverse CDF to these numbers
random_variables = [inverse_cdf(u) for u in uniform_random_numbers]

fig = px.histogram(x=random_variables).update(
    layout=dict(
        height=500,
        width=800,
        margin=dict(t=60, b=20, l=20, r=20),
    )
)

fig.write_image("probability-distribution-of-tunnel-cost-overruns-histogram.svg")

In [343]:
# Define the range of x-values to use in the distributions
cdf_x = np.linspace(df["CostFactor"].min(), df["CostFactor"].max(), 100)

# Fit the data to 5 different distributions using the scipy library
# For example: `stats.gamma.fit()` fits the data and provides the associated
# parameters for the `gamma` distribution function. `stats.gamma.cdf` uses the
# parameters and the x-range to calculate an array that represents the CDF
cdf_gamma = stats.gamma.cdf(cdf_x, *stats.gamma.fit(random_variables))
cdf_weibull_min = stats.weibull_min.cdf(cdf_x, *stats.weibull_min.fit(random_variables))
cdf_weibull_max = stats.weibull_max.cdf(cdf_x, *stats.weibull_max.fit(random_variables))
cdf_exponweib = stats.exponweib.cdf(cdf_x, *stats.exponweib.fit(random_variables))

# Add the CDFs to the original scatter plot
for name, trace in {
    "gamma": cdf_gamma,
    "weibull_min": cdf_weibull_min,
    "weibull_max": cdf_weibull_max,
    "exponweib": cdf_exponweib,
}.items():
    fig_cdf.add_trace(go.Scatter(x=cdf_x, y=trace, name=name, mode="lines"))

fig_cdf.update(layout=dict(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)))

fig_cdf.write_image("probability-distribution-of-tunnel-cost-overruns-cdf-dists.svg")
fig_cdf.show()

In [344]:
# Calculate the distribution expected value and 95% confidence interval
expected_value = stats.gamma(*stats.gamma.fit(random_variables)).expect()

# Define the range of confidence intervals to use
ci_range = np.linspace(0.5, 0.99)

# Calculate the confidence intervals using the `gamma.fit` method and the same
# `random_varialbes` used previously
ci_vals = [
    (i, *stats.gamma(*stats.gamma.fit(random_variables)).interval(i)) for i in ci_range
]

# Make a `dataframe` and add the expected value column for plotting
df = pd.DataFrame(ci_vals, columns=["CI", "Lower", "Upper"])
df["Expected Value"] = expected_value

# Plot the figure
fig = (
    px.line(df, x="CI", y=["Lower", "Upper", "Expected Value"])
    .update(
        layout=dict(
            title="<b>Confidence interval and cost factor</b><br><i>Fixed Links (bridges and tunnel)</i>",
            height=500,
            width=800,
            margin=dict(t=60, b=20, l=20, r=20),
            legend=dict(title="Bound", yanchor="top", y=0.99, xanchor="left", x=0.01),
            xaxis=dict(title="Confidence Interval"),
            yaxis=dict(title="Cost Factor"),
        )
    )
    .add_annotation(
        text=f"<b>Expected Value = {round(expected_value, 2)}</b>",
        showarrow=False,
        xref="paper",
        x=0.75,
        yref="paper",
        y=0.25,
    )
)

fig.write_image("probability-distribution-of-tunnel-cost-overruns-confidence.svg")
fig.show()